In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
import joblib
from sklearn.model_selection import GridSearchCV

In [1]:
import pandas as pd
df = pd.read_csv('data/combine.csv')
df.head()

,RequirementSource,ContainsRelevantRegulation,User,DocumentID,Title,RegulatorId,SourceLanguage,DocumentTypeId,PublicationDate,IsPdf,Content
0,Regulation,True,00002B53,99f7-77d9-4c72-a45a-049dd24b,"Circular Date: March 28, 2023 Department: LIST...",fd04-1577-4e11-80de-65302691,English,27fd-37bf-4c5e-94ec-8c7528bd,2023-03-28 16:05:00+00:00,True,"Circular Date: March 28, 2023 Department: LIST..."
1,Regulation,False,00002B53,104e-8d13-4537-8e56-415dacd8,"Ref. No.: LIC/SE/2022-23/157 Date: 11th March,...",76d2-78ec-48b7-a01e-7a8dd5c5,English,ee59-4ede-4bba-90a4-45a510f9,2023-03-11 15:00:00.013333+00:00,True,"Ref. No.: LIC/SE/2022-23/157 Date: 11th March,..."
2,Regulation,False,0000164E,b634-3641-4f16-96cc-99e1cb85,Bundesgesetzblatt Part I 2024 Issued to Bonn o...,9a27-032b-4be1-a886-44b1e915,German,e27a-8235-4bf6-96f0-3f56d2a4,2024-05-27 09:51:00.013000+00:00,True,Bundesgesetzblatt Part I 2024 Issued to Bonn o...
3,Guidance,False,0000164E,b634-3641-4f16-96cc-99e1cb85,Bundesgesetzblatt Part I 2024 Issued to Bonn o...,9a27-032b-4be1-a886-44b1e915,German,e27a-8235-4bf6-96f0-3f56d2a4,2024-05-27 09:51:00.013000+00:00,True,Bundesgesetzblatt Part I 2024 Issued to Bonn o...
4,Guidance,False,000006A5,f452-1e80-482e-ab6b-86d02faf,Women from rural areas will receive scholarshi...,604b-bb64-43cd-b17c-ba097ac6,Spanish,9e76-0039-4827-a596-e841437e,2022-03-05 18:20:00.023333+00:00,False,"\n<meta http-equiv=""Content-Type"" content=""tex..."


In [2]:
tope = df['User'].value_counts().reset_index()
tope.columns = ['User', 'Count']
filtered_df = df[df['ContainsRelevantRegulation'] == True]
user_counts_df = filtered_df['User'].value_counts().reset_index()
user_counts_df.columns = ['User', 'Count']
merged_user = pd.merge(user_counts_df, tope, on='User', suffixes=('_user_relevant', '_user_submited'))
merged_user['Percentage'] = (merged_user['Count_user_relevant'] / merged_user['Count_user_submited']) * 100
merged_user = merged_user.sort_values(by='Percentage', ascending=False)
print(merged_user.shape)
merged_user

(128, 4)


,User,Count_user_relevant,Count_user_submited,Percentage
73,0000077A,15,15,100.000000
120,000006FB,1,1,100.000000
56,00000BFA,22,22,100.000000
81,00002E2C,11,11,100.000000
61,00000A29,18,18,100.000000
...,...,...,...,...
41,000006A5,51,16586,0.307488
105,00000706,3,1261,0.237906
124,000006D6,1,532,0.187970
117,000007A1,1,556,0.179856


In [3]:
tope_2 = df['DocumentID'].value_counts().reset_index()
tope_2.columns = ['DocumentID', 'Count']
filtered_df = df[df['ContainsRelevantRegulation'] == True]
DocumentID_counts_df = filtered_df['DocumentID'].value_counts().reset_index()
DocumentID_counts_df.columns = ['DocumentID', 'Count']
merged_DocumentID = pd.merge(DocumentID_counts_df, tope_2, on='DocumentID', suffixes=('_document_counts_relevant', '_document_submited'))
merged_DocumentID['Percentage'] = (merged_DocumentID['Count_document_counts_relevant'] / merged_DocumentID['Count_document_submited']) * 100
merged_DocumentID = merged_DocumentID.sort_values(by='Count_document_counts_relevant', ascending=False)
print(merged_DocumentID.shape)
merged_DocumentID.tail(20)

(8735, 4)


,DocumentID,Count_document_counts_relevant,Count_document_submited,Percentage
4223,bb4c-0be5-40f4-adba-ea4ea4a8,1,2,50.0
4222,dddb-478e-4369-b3f3-7a51e1ec,1,2,50.0
4221,ae25-2cef-421a-9ad1-2de16ff8,1,2,50.0
4220,0d13-5e67-4705-8df8-8c1e7e37,1,1,100.0
4219,1e63-37e6-46fb-b98d-53bc6c2d,1,1,100.0
4218,237c-1b6e-4cb5-9483-574dd504,1,2,50.0
4217,25df-1425-468a-ba3e-b99716c5,1,1,100.0
4215,623b-ada7-4da4-a0e8-a8d9b871,1,2,50.0
4204,441d-1971-46d1-887d-5eaf7db6,1,2,50.0
4214,d24a-893f-4658-a727-5c27f600,1,1,100.0


In [4]:
tope_3 = df['DocumentTypeId'].value_counts().reset_index()
tope_3.columns = ['DocumentTypeId', 'Count']

tope_3 = tope_3.sort_values(by='Count', ascending=False)
filtered_df = df[df['ContainsRelevantRegulation'] == True]
Documenttype_counts_df = filtered_df['DocumentTypeId'].value_counts().reset_index()
Documenttype_counts_df.columns = ['DocumentTypeId', 'Count']
merged_Documenttype = pd.merge(Documenttype_counts_df, tope_3, on='DocumentTypeId', suffixes=('_document_type_relevant', '_documenttype_submited'))
merged_Documenttype.head()
merged_Documenttype['Percentage'] = (merged_Documenttype['Count_document_type_relevant'] / merged_Documenttype['Count_documenttype_submited']) * 100
merged_Documenttype = merged_Documenttype.sort_values(by='Percentage', ascending=False)
print(merged_Documenttype.shape)
merged_Documenttype.head(20)

(86, 4)


,DocumentTypeId,Count_document_type_relevant,Count_documenttype_submited,Percentage
71,45cb-dd07-4a3e-8ad2-f22ad5e3,3,3,100.000000
48,6dbd-8a10-41c2-840f-b7d7dc64,15,16,93.750000
34,3471-e72c-4cdd-9632-ee3af7a4,40,61,65.573770
60,4e5b-3466-4cb1-be02-cad95837,8,13,61.538462
43,61d0-0ee6-43b4-82bc-bc1e9229,17,28,60.714286
54,58d5-ef6d-4ad5-a7cd-6cb58906,12,21,57.142857
25,21c8-83c8-4d29-afa2-27e74baa,104,197,52.791878
30,3543-83c9-491d-9f0a-2bfdda57,69,141,48.936170
32,187e-522c-48af-a724-30101382,50,103,48.543689
22,fa5c-4e4f-4fa3-ad2f-80af70c6,114,308,37.012987


In [5]:
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Initialize stopwords
stop_words = set(stopwords.words('english'))

def preprocess(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
        # Tokenize
        words = word_tokenize(text)
        # Remove stopwords
        words = [word for word in words if word not in stop_words and not word.isdigit()]
        return words
    return []


[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [6]:
df['processed_title'] = df['Title'].apply(preprocess)

In [7]:
df['processed_title'] = df['Title'].apply(preprocess)
all_words = [word for sublist in df['processed_title'] for word in sublist]
word_freq = Counter(all_words)
top_100_keywords = word_freq.most_common(100)
keywords_list = [keyword for keyword, freq in top_100_keywords]

In [8]:
relevant_document_ID = merged_DocumentID['DocumentID'].head(100).tolist()

In [9]:
relevant_User = merged_user['User'].head(100).tolist()

In [10]:
relevant_document_type = merged_Documenttype['DocumentTypeId'].head(100).tolist()

In [14]:
print(len(relevant_document_type))
print(len(keywords_list))
print(len(relevant_User))
print(len(relevant_document_ID))

86
100
100
100


In [61]:
def is_relevant(document):
    # Check regulator
#     if document['DocumentID'] in relevant_document_ID :
#         return True
    # Check document type
    if document['DocumentTypeId'] in relevant_document_type and document['User'] in relevant_User and document['DocumentID'] in relevant_document_ID:
        return True
#     # Check user
#     if document['User'] in relevant_User:
#         return True
#     Check keywords in title, handling non-string values
#     title = str(document['Title']).lower() if pd.notna(document['Title']) else ''
#     if any(keyword in title for keyword in keywords_list):
#         return True
    return False


In [79]:
def is_relevant(document):
    conditions_met = 0

    # Check if the document ID is relevant
    if document['DocumentID'] in relevant_document_ID:
        conditions_met += 1

    # Check if the document type is relevant
    if document['DocumentTypeId'] in relevant_document_type:
        conditions_met += 1

    # Check if the user is relevant
    if document['User'] in relevant_User:
        conditions_met += 1

    # Check if the title contains relevant keywords
#     title = str(document['Title']).lower() if pd.notna(document['Title']) else ''
#     if any(keyword in title for keyword in keywords_list):
#         conditions_met += 1

    # Return True if at least two conditions are met
    return conditions_met >= 2


In [80]:
df['IsRelevant'] = df.apply(is_relevant, axis=1)

In [81]:
df['IsRelevant'].unique()

array([ True, False])

In [82]:
print(df['IsRelevant'].value_counts())
print(df['ContainsRelevantRegulation'].value_counts())

True     91031
False    47678
Name: IsRelevant, dtype: int64
False    123097
True      15612
Name: ContainsRelevantRegulation, dtype: int64


In [66]:
df.sample(10)

,RequirementSource,ContainsRelevantRegulation,User,DocumentID,Title,RegulatorId,SourceLanguage,DocumentTypeId,PublicationDate,IsPdf,Content,processed_title,IsRelevant
112513,Guidance,False,000006A5,69b0-548d-4581-9240-d9714725,"\n<meta http-equiv=""Content-Type"" content=""tex...",ae17-97ef-4a38-9fa6-0420ec49,Spanish,755e-aec5-4995-9047-aaeb9074,2021-10-29 17:28:35.843333+00:00,False,"\n<meta http-equiv=""Content-Type"" content=""tex...","[meta, httpequivcontenttype, contenttexthtml, ...",False
101062,Regulation,False,000011C0,e5eb-313a-4649-8d93-d0240bae,Resolution No 115/023 new system,ca5c-40db-4126-a6bd-7f0442c8,Spanish,9d3a-714e-4ddf-b821-61fd69ea,2023-07-12 16:21:00.010000+00:00,False,<div> The registration of the database named C...,"[resolution, new, system]",False
105568,Regulation,False,1FFEEBD9,fe39-f02a-45c6-b5ed-06683dd5,First Chamber of the States General 1 Year of ...,8dc5-eb07-490c-ba0f-81a3814c,Flemish,e27a-8235-4bf6-96f0-3f56d2a4,2024-06-04 08:05:59.980000+00:00,True,First Chamber of States General 1 Year of sitt...,"[first, chamber, states, general, year, sittin...",False
23479,Regulation,False,000006A5,8183-3830-48b9-985d-846143a1,Digitally signed by: JORGE ALBERTO ORTECHO RED...,a501-ebb6-4f34-beca-df97a22c,Spanish,ebf3-0770-48fa-91f6-715a6c31,2021-10-29 17:28:35.843333+00:00,True,Digitally signed by: JORGE ALBERTO ORTECHO RED...,"[digitally, signed, jorge, alberto, ortecho, r...",False
13294,Guidance,False,00002A96,30c2-b2b3-4cd6-9773-2224e410,Determinants of capital market dynamics,ddd5-26d9-4ccb-9737-ba639c4b,Spanish; Castilian,84e1-0770-4ddc-907c-37373f70,2024-06-13 14:41:00.010000+00:00,True,<p>Working Papers of the Board of Directors of...,"[determinants, capital, market, dynamics]",False
92214,Guidance,False,8DBE7FF8,b12d-64ab-439e-84d4-b656b500,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,&quot;Feb...",867e-1977-401f-9522-5c9c3145,Spanish,a3c7-44e8-436a-9d43-1968869c,2024-03-15 12:06:00+00:00,True,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, &quot;Fe...","[quotfebruary, 2024quotquotfebruary, 2024quot]",False
135381,Regulation,False,00002ED8,391c-b7af-4f04-a82b-d2e5e968,Weekly Economic Indicators 03rd September 2021...,6314-5cd6-4ed8-90a6-ebab0c92,English,d73b-5443-4fe0-800c-c88fbc76,2021-10-29 17:28:35.843333+00:00,True,Weekly Economic Indicators 03rd September 2021...,"[weekly, economic, indicators, 03rd, september...",False
99281,Guidance,False,00000E6A,6ef4-20f5-4bf0-9306-98cec907,ADDENDUM TO PACKAGE LEAFLET SUPPLEMENT S.A. SA...,749f-7ebf-4d0f-8ed9-dc645d9b,Spanish,27fd-37bf-4c5e-94ec-8c7528bd,2024-07-30 08:35:59.996000+00:00,True,<p>ADDENDA TO PACKAGE-LEAFLET SUPPLEMENT S.A. ...,"[addendum, package, leaflet, supplement, sa, s...",False
48943,Regulation,True,00000E97,dd2a-4807-48c1-b12c-9b70482e,On ensuring access of the National Agency for ...,1ec1-7d3e-45ab-bd4f-ea7072a5,Ukrainian,e27a-8235-4bf6-96f0-3f56d2a4,2024-06-11 15:10:59.990000+00:00,False,"<div> <link href=""https://zakon.rada.gov.ua/la...","[ensuring, access, national, agency, preventio...",False
43980,Regulation,False,00002DF8,2eeb-28ca-4bf3-bc55-520df5d1,COVID-19 (ELECTION OF JURATS) (JERSEY) REGULAT...,7147-8888-46d6-806a-9e0dfec4,English,9d3a-714e-4ddf-b821-61fd69ea,2021-10-29 17:28:35.843333+00:00,False,<p><strong>COVID-19 (ELECTION OF JURATS) (JERS...,"[covid19, election, jurats, jersey, regulations]",False


In [52]:
df['DocumentID'].unique().shape

(77721,)

In [83]:
import pandas as pd
df = pd.read_csv('data/regulations.csv')
df.head()

,Title,RegulatorId,SourceLanguage,DocumentTypeId,PublicationDate,IsPdf,Content,DocumentID
0,Federal Law Gazette Part I 2024 Issued to Bonn...,9a27-032b-4be1-a886-44b1e915,German,e27a-8235-4bf6-96f0-3f56d2a4,2024-08-02 08:45:59.993000+00:00,True,Federal Gazette Part I 2024 Issued to Bonn on ...,3d80-b1ff-4b76-b59a-8c1be8a2
1,Readout of Vice President Harris’s Call with Y...,d47a-5966-4cac-802f-00b7cf36,English,9e76-0039-4827-a596-e841437e,2024-08-02 00:01:00.003000+00:00,False,"<time>August 01, 2024</time><div> <div> <p>Vic...",4b15-5b76-4828-b3b3-f3a58402
2,NOTE # 13.344 01 August 2024 The Agents are in...,749f-7ebf-4d0f-8ed9-dc645d9b,Spanish,f6e0-d6b3-471f-9811-dce56652,2024-08-02 05:51:00+00:00,True,NOTE # 13.344 01 August 2024 The Agents are in...,a6ec-8a09-443d-b28f-34763ca6
3,NOTE # 13.344 01 August 2024 The Agents are in...,749f-7ebf-4d0f-8ed9-dc645d9b,Spanish,f6e0-d6b3-471f-9811-dce56652,2024-08-02 05:51:00+00:00,True,NOTE # 13.344 01 August 2024 The Agents are in...,a6ec-8a09-443d-b28f-34763ca6
4,NOTE # 13.344 01 August 2024 The Agents are in...,749f-7ebf-4d0f-8ed9-dc645d9b,Spanish,f6e0-d6b3-471f-9811-dce56652,2024-08-02 05:51:00+00:00,True,NOTE # 13.344 01 August 2024 The Agents are in...,a6ec-8a09-443d-b28f-34763ca6


In [ ]:
df['processed_content'] = df['Content'].apply(preprocess)
all_words = [word for sublist in df['processed_content'] for word in sublist]
word_freq = Counter(all_words)
top_100_keywords = word_freq.most_common(40)
keywords_list = [keyword for keyword, freq in top_100_keywords]

In [89]:
def is_relevant(document):
    conditions_met = 0

    # Check if the document ID is relevant
    if document['DocumentID'] in relevant_document_ID:
        conditions_met += 1

    # Check if the document type is relevant
    if document['DocumentTypeId'] in relevant_document_type:
        conditions_met += 1

#     # Check if the user is relevant
#     if document['User'] in relevant_User:
#         conditions_met += 1

#     Check if the title contains relevant keywords
#     title = str(document['Title']).lower() if pd.notna(document['Title']) else ''
#     if any(keyword in title for keyword in keywords_list):
#         conditions_met += 1

    # Return True if at least two conditions are met
    return conditions_met >= 2

In [90]:
df['IsRelevant'] = df.apply(is_relevant, axis=1)

In [91]:
print(df['IsRelevant'].value_counts())
print(df['ContainsRelevantRegulation'].value_counts())

False    132600
True       1483
Name: IsRelevant, dtype: int64


KeyError: 'ContainsRelevantRegulation'